In [10]:
import re 
import json
#import grequests
import requests
from bs4 import BeautifulSoup
from datetime import datetime 
import os
import pandas as pd
import shutil
import yaml
import pickle
#from helium import *
import time
import traceback
from agenda_tools import get_schedule, download_images, get_category
from requests_html import HTMLSession
from IPython.display import Image
import urllib

session = HTMLSession()

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from agenda_tools.get_schedule import remove_blank_spaces
from selenium.webdriver.chrome.service import Service

In [18]:
from selenium import webdriver


options = webdriver.ChromeOptions()
#options.add_argument('--private')
#options.add_argument('--no-sandbox')
#options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 12.2; rv:97.0) Gecko/20100101 Firefox/97.0')
#options.add_argument("--headless")
driver = webdriver.Chrome(executable_path='/home/cesarppz/Documents/jobs/web_scraping/javier/agenda/driver/chromedriver', options=options)

WebDriverException: Message: unknown error: Chrome failed to start: crashed.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)


In [ ]:
url = 'google.com'

In [ ]:
driver.get(url)

In [10]:
driver.find_element_by_xpath('//*[text()="¿CÓMO CONTACTARNOS?"]/../following-sibling::p/a[1]').text

'(+57) 1 508 7662'

In [11]:
def get_header_(link,cookie='SPSI=f6dc1b0fa852702878d8c2c22654faa5; SPSE=NuTmdUNxZ13k3xGyyhZq+v+qqgs+NvAtfbD4Fb4RqC+jvFRAsZKkeN7S5nXVpN54pAJI66Yiuzgk6LPth/dopg==; UTGv2=D-h4a224b09e681c211039b9d483d4b8db0d94; PHPSESSID=8ea7994c69df3038cff41faef1de5d3f; spcsrf=457576b2844dc821551e6f50010a6f6a; sbtsck=javgfZ3MY8bP1OX2xhMpr3PEW8UeEjrBn6ebvU6P4d64+c=; PRLST=; adOtr=1c6bffda058; sp_lit=RJQh8O1pL4J+SXXH9McHzw=='):
    headers = {
  'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
  'Accept-Language': 'en-US,en;q=0.5',
  'content-type': 'text/html; charset=UTF-8',
#   'x-window-id': 'window-jvhvUWAfCvwZIJ3--Jdq3',
#   'x-browser-id': 'browser-gBb1AiFQ6RtKtXpZyA1A8',
#   'apollographql-client-name': 'core-front-browser',
#   'Origin': 'https://www.thefork.es',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'none',
  'Sec-Fetch-User':'?1',
  'Referer': link,
  'Connection': 'keep-alive',
  'Cookie': cookie ,
  'TE': 'trailers'
}
    return headers

In [15]:
header = get_header_('https://www.thefork.es/restaurante/wok-4-you-bravo-murillo-r677423',cookie='datadome=.96Md~SqfKt1u3PgKi5PfUW_7Ouo.BmWkU5eI1xvtg0zQ.~7Z5aPa8Bv4UuJXLwik1NNMdQbsfVbOlbQxzwJUnWnM8_NjE4VgWyQz5StO33_r2uVVq-iQrwvvY~aaB6g; connected=false; _evidon_consent_cookie={"consent_date":"2021-10-09T01:23:42.231Z","consent_type":1}; tf_ab_test=filters_with_push_dhp%3Dcontrol; CC=15101-cfd; source_code=2022-02-24T19:23:45||direct|15101-cfd|; G_ENABLED_IDPS=google; __stripe_mid=c9b392a7-4200-476d-bbbb-3f73e42dae3aaec5e2; tf_session=session-1zeUNNQCT84vhQVDsyLUy; trackingId=0a19e961-e310-4abd-9571-e39b201d6b65; tf_evidon_consent={%22advertising%22:true%2C%22analytics%20&%20customisation%22:true}; tf_abtests_freshness=true')

In [17]:
def make_request(link,headers):
    r = requests.get(link,headers=headers)
    if r.status_code == 200:
        return r, True
    else:
        return _, False

In [20]:
with open('../restaurant/fork.dat','rb') as f:
    info_r = pickle.load(f)

In [41]:
def get_menu_re(link_menu,header):
    succesfull_request = False
    menu_text = []
    menu_price = []
    while succesfull_request == False:
        request_menu, succesfull_request = make_request(link_menu,header)


        if succesfull_request:
            s = BeautifulSoup(request_menu.text,'lxml')
            big_menu = s.find_all('dl',attrs={'class':"css-1ndrudv e1xxesyf0"})
            if big_menu == []:
                print('Error big menu')
            for i in big_menu:
                for j in i.find_all('div',attrs={'class':"css-1iqq28q elkhwc30"}):
                    try:
                        menu_text.append(j.span.text)
                        menu_price.append(j.dd.text.replace('\xa0',''))
                    except:
                        pass

        else:
            print(f'Error en el indice {title} / {link_menu}')
            cookie = str(input('cookie : ')).replace(' Cookie 	','',1)
            if cookie == 'p':
                break
            header = get_header_(link_menu,cookie)

    menu = [' -- '.join(i) for i in list(zip(menu_text,menu_price))]
    return menu, header

In [21]:
info_r[0]

<Response [200]>

In [38]:
data = []
cookie = 'datadome=.96Md~SqfKt1u3PgKi5PfUW_7Ouo.BmWkU5eI1xvtg0zQ.~7Z5aPa8Bv4UuJXLwik1NNMdQbsfVbOlbQxzwJUnWnM8_NjE4VgWyQz5StO33_r2uVVq-iQrwvvY~aaB6g; connected=false; _evidon_consent_cookie={"consent_date":"2021-10-09T01:23:42.231Z","consent_type":1}; tf_ab_test=filters_with_push_dhp%3Dcontrol; CC=15101-cfd; source_code=2022-02-24T19:23:45||direct|15101-cfd|; G_ENABLED_IDPS=google; __stripe_mid=c9b392a7-4200-476d-bbbb-3f73e42dae3aaec5e2; tf_session=session-1zeUNNQCT84vhQVDsyLUy; trackingId=0a19e961-e310-4abd-9571-e39b201d6b65; tf_evidon_consent={%22advertising%22:true%2C%22analytics%20&%20customisation%22:true}; tf_abtests_freshness=true'
link = 'https://www.thefork.es/restaurante/wok-4-you-bravo-murillo-r677423'
header = get_header_(link,cookie)
for idx, r in enumerate(info_r[:2]):
    print(f'Scrapig {idx+1}/{len(info_r)}')
    header, data_info = get_info(r,header)
    data.append(data_info)
    

Scrapig 1/1425
https://www.thefork.es/restaurante/diurno-madrid-r30520/menu
Error en menu:  'str' object has no attribute 'items'
Scrapig 2/1425


Traceback (most recent call last):
  File "<ipython-input-37-1673df3a4145>", line 28, in get_info
    manu, header = get_menu_re(link_menu,cookie)
  File "<ipython-input-34-017849151d75>", line 5, in get_menu_re
    request_menu, succesfull_request = make_request(link_menu,header)
  File "<ipython-input-17-791e858fba0d>", line 2, in make_request
    r = requests.get(link,headers=headers)
  File "/home/cesar/anaconda3/envs/ws/lib/python3.7/site-packages/requests/api.py", line 75, in get
    return request('get', url, params=params, **kwargs)
  File "/home/cesar/anaconda3/envs/ws/lib/python3.7/site-packages/requests/api.py", line 61, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/cesar/anaconda3/envs/ws/lib/python3.7/site-packages/requests/sessions.py", line 515, in request
    prep = self.prepare_request(req)
  File "/home/cesar/anaconda3/envs/ws/lib/python3.7/site-packages/requests/sessions.py", line 453, in prepare_request
    hooks=merge_hooks(r

https://www.thefork.es/restaurante/mercado-de-la-reina-r6882/menu
Error en menu:  'str' object has no attribute 'items'


Traceback (most recent call last):
  File "<ipython-input-37-1673df3a4145>", line 28, in get_info
    manu, header = get_menu_re(link_menu,cookie)
  File "<ipython-input-34-017849151d75>", line 5, in get_menu_re
    request_menu, succesfull_request = make_request(link_menu,header)
  File "<ipython-input-17-791e858fba0d>", line 2, in make_request
    r = requests.get(link,headers=headers)
  File "/home/cesar/anaconda3/envs/ws/lib/python3.7/site-packages/requests/api.py", line 75, in get
    return request('get', url, params=params, **kwargs)
  File "/home/cesar/anaconda3/envs/ws/lib/python3.7/site-packages/requests/api.py", line 61, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/cesar/anaconda3/envs/ws/lib/python3.7/site-packages/requests/sessions.py", line 515, in request
    prep = self.prepare_request(req)
  File "/home/cesar/anaconda3/envs/ws/lib/python3.7/site-packages/requests/sessions.py", line 453, in prepare_request
    hooks=merge_hooks(r

In [47]:
link_menu = 'https://www.thefork.es/restaurante/diurno-madrid-r30520/menu'
link_menu

'https://www.thefork.es/restaurante/diurno-madrid-r30520/menu'

In [44]:
request_menu, succesfull_request = make_request(link_menu,header)

In [51]:
s.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html lang="es-ES">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge,chrome=1" http-equiv="x-ua-compatible"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="no" name="msapplication-tap-highlight"/>
<script>
if (!Object.assign) { document.write('<script crossorigin="anonymous" src="//polyfill.io/v3/polyfill.min.js" />'); }
if (!window.Intl) { document.write('<script src="//polyfill.io/v3/polyfill.min.js?features=Object.values,Intl.~locale.es-ES"/>'); }
</script>
<script>
window.SSR = {"request_id":"6ce0fd79-f5cc-44e9-abad-38f459c318d5","experimentMismatchDebugData":{"experiments":{"filters_with_push_dhp":"control"},"freshness":"true"}}
</script>
<script>document.documentElement.dataset.js = 'on';</script>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam-cell.nr-data

In [45]:
s = BeautifulSoup(request_menu.text,'lxml')

In [53]:
s.find_all('h3')

[<h3 class="css-1kcpov2 e7dhrrp0" color="special.white">Reservation</h3>,
 <h3 class="css-b3f6t9 e7dhrrp0"><span>Buscar restaurante</span></h3>]

In [42]:
get_menu_re(link_menu,header)

Error big menu


([],
 {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:96.0) Gecko/20100101 Firefox/96.0',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
  'Accept-Language': 'en-US,en;q=0.5',
  'Content-Type': 'application/json',
  'x-thefork-product-name': 'core-front-browser',
  'x-window-id': 'window-jvhvUWAfCvwZIJ3--Jdq3',
  'x-browser-id': 'browser-gBb1AiFQ6RtKtXpZyA1A8',
  'apollographql-client-name': 'core-front-browser',
  'Origin': 'https://www.thefork.es',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'none',
  'Sec-Fetch-User': '?1',
  'Referer': 'https://www.thefork.es/restaurante/wok-4-you-bravo-murillo-r677423',
  'Connection': 'keep-alive',
  'Cookie': 'datadome=.96Md~SqfKt1u3PgKi5PfUW_7Ouo.BmWkU5eI1xvtg0zQ.~7Z5aPa8Bv4UuJXLwik1NNMdQbsfVbOlbQxzwJUnWnM8_NjE4VgWyQz5StO33_r2uVVq-iQrwvvY~aaB6g; connected=false; _evidon_consent_cookie={"consent_date":"2021-10-09T01:23:42.231Z","consent_type"

In [24]:
pd.read_excel('/home/cesar/Documents/job/web_scraping/cg_wilson/edarabia/results_edarabia_5_3.xlsx').Website

0                           http://www.sunmarkedubai.com
1                      http://www.reachbritishschool.com
2                                 http://www.hisdubai.ae
3                             http://www.ebi-clermont.fr
4      http://www.richmondparkacademy.org/?utm_source...
                             ...                        
191                         http://www.newhorizon-eg.com
192                              http://www.mescairo.com
193         http://www.nefertariinternationalschools.com
194                             http://www.mbisegypt.com
195                               http://www.daisegy.com
Name: Website, Length: 196, dtype: object

In [2]:
cd ../pipeline_agenda/

/home/cesar/Documents/job/web_scraping/javier/pipeline_agenda


In [3]:
from cancat import *

In [4]:
concat_dataframes('/home/cesar/Desktop/javier_fiverr/')

No columns to parse from file Sitio results_conde_duque_santa_2_3.csv
No columns to parse from file Sitio results_conde_duque_alberto_2_3.csv


In [5]:
concat_images('/home/cesar/Desktop/javier_fiverr/')

In [4]:
requests.get('https://www.edarabia.com/schools/united-states/')

<Response [200]>

In [2]:
pd.read_excel('/home/cesar/Documents/job/web_scraping/cg_wilson/edarabia/results_edarabia_5_3.xlsx')

,school_name,Leadership,Address,Tel,Website
0,Sunmarke School Dubai,Dr. Neil Hopkin FRSA (Executive Principal),"Jumeirah Village Triangle - Dubai , UAE",Tel: +97144238900,http://www.sunmarkedubai.com
1,Reach British School Abu Dhabi,Mr. Dene Bright (Principal),"Baniyas East - Abu Dhabi , UAE",Tel: +97125822030,http://www.reachbritishschool.com
2,Horizon International School,Mr. Darren Gale (Director),"Street 9A, Off Al Wasl Road, Umm Al Sheif - D...",Tel: +97143483314,http://www.hisdubai.ae
3,Ecole Bilingue Internationale,Pascale Verniere (Principal),"5, rue Bansac - Clermont Ferrand, France ...",Tel: +33473980973,http://www.ebi-clermont.fr
4,Richmond Park Academy,Paul Mundy-Castle (Principal),"Park Avenue, East Sheen - Richmond upon Thame...",Tel: +442088768891,http://www.richmondparkacademy.org/?utm_source...
...,...,...,...,...,...
191,New Horizon International School,Mrs. Aicha Wassef (Director),"8 Al Ofouk Street, Zahraa Al Maadi City – Sect...",Tel: +20227332058,http://www.newhorizon-eg.com
192,"Modern English School, Cairo",Mrs. Nicola Singleton (Director),"New Cairo, South of Police Academy, - Cairo ,...",Tel: +2019836,http://www.mescairo.com
193,Nefertari British International School,Mrs. Chairperson: Nadia Hafez,"K.M. 22, Cairo - Ismailia Desert Road left sid...",Tel: +20226573978,http://www.nefertariinternationalschools.com
194,Maadi British International School,Mr. James McDonald (Principal),"4th District, Zahraa Al Maadi - Cairo , Egypt",Tel: +20225178288 / +20225178301,http://www.mbisegypt.com


# Proxy

In [95]:
with open('/home/cesar/Downloads/good_proxies.txt','r') as f:
    lines = f.readlines()

In [3]:
list_cities = ['https://www.edarabia.com/schools/abu-dhabi/','https://www.edarabia.com/schools/ajman/','https://www.edarabia.com/schools/al-ain/','https://www.edarabia.com/schools/dubai/','https://www.edarabia.com/schools/fujairah/',
'https://www.edarabia.com/schools/rak/','https://www.edarabia.com/schools/sharjah/']

In [9]:
requests.get(list_cities[0])

<Response [200]>

In [13]:
list_cities[0]

'https://www.edarabia.com/schools/abu-dhabi/'

In [4]:
first_page_pattern = re.compile(r'.*?pg=1')

In [14]:
pattern_city = re.compile('https:.*/schools/(.*)/.*')

In [35]:
a = 'https://www.edarabia.com/schools/rak/?pg=2'
if re.match(first_page_pattern,a):
    print('a')

In [36]:
re.match(pattern_city,a).group(1)

'rak'

In [19]:
def get_page(last_page,idx_p, city):
    data = []
    for i in range(1,last_page+1):
        print('\t Page Index:',i)
        for second_proxy in lines:
            link = '{}?pg={}'.format(city,i)
            if re.match(first_page_pattern,link):
                link = '{}'.format(city)
           # print('\t Link Page:',link)
            try:
                rq = requests.get(link,proxies={'http':second_proxy,'https':second_proxy},timeout=3)
                soup_s = BeautifulSoup(rq.text,'lxml')
                print('\t',rq.status_code)
                if soup_s.find_all('div',attrs={'class':"list-items"}):
                    list_links = [i.find('div',attrs={'class':"row"}).find('div',attrs={'class':"col-md-10 col-sm-10 col-xs-10"}) for i in soup.find_all('div',attrs={'class':"list-items"}) if i is not None]
                    links_page = [i.h5.a.get('href') for i in list_links if i is not None]
                    data.append(links_page)
                    print('Success')
                    break
            except ConnectionError as cx:
                pass
            except Exception as ex:
                print('\tFailed second round')
                print(ex)
    return data

In [21]:
# Extraer link con ciudades
data_links = []
for city in list_cities:
    print(city)
    for idx_p, proxy in enumerate(lines):
        try:
            r = requests.get(city,proxies={'http':proxy,'https':proxy},timeout=3)
            soup = BeautifulSoup(r.text,'lxml')
            print(r.status_code)
            print(soup.find('a',attrs={'title':"Last"}))
            if soup.find('a',attrs={'title':"Last"}):
                last_page = int(soup.find('a',attrs={'title':"Last"}).get('data-page'))
                print(last_page)
                data = get_page(last_page,idx_p, city)
                data_dict = {'City':re.match(pattern_city,city).group(1),'Data':data}
                data_links.append(data_dict)
                break
                
        except ConnectionError as cx:
            pass
        except Exception as ex:
            print('Failed')
            print(ex)

https://www.edarabia.com/schools/abu-dhabi/
200
None
200
<a data-page="6" href="#" title="Last">Last</a>
6
	 Page Index: 1
	 200
Success
	 Page Index: 2
	 200
Success
	 Page Index: 3
	 200
Success
	 Page Index: 4
	 200
Success
	 Page Index: 5
	 200
	 200
Success
	 Page Index: 6
	 200
Success
https://www.edarabia.com/schools/ajman/
200
None
200
None
200
None
200
None
200
<a data-page="2" href="#" title="Last">Last</a>
2
	 Page Index: 1
	 200
Success
	 Page Index: 2
	 200
Success
https://www.edarabia.com/schools/al-ain/
200
None
200
None
200
<a data-page="3" href="#" title="Last">Last</a>
3
	 Page Index: 1
	 200
	 200
	 200
	 200
	 200
	 200
	 200
	 200
Success
	 Page Index: 2
	 200
	 200
	 200
	 200
	 200
Success
	 Page Index: 3
	 200
Success
https://www.edarabia.com/schools/dubai/
200
None
200
None
200
None
200
None
200
None
200
<a data-page="9" href="#" title="Last">Last</a>
9
	 Page Index: 1
	 200
Success
	 Page Index: 2
	 200
Success
	 Page Index: 3
	 200
Success
	 Page Index: 4
	 2

In [3]:
data_links = []

for page in range(1,23):
    for i in lines:
        i = i.replace('\n','')
        proxy = i
        link = 'https://www.edarabia.com/schools/uae/?pg={}'.format(page)
        try:
            r = requests.get(link,proxies={'http':proxy,'https':proxy},timeout=3)
            soup = BeautifulSoup(r.text,'lxml')
            print(r.status_code)
            if soup.find_all('div',attrs={'class':"list-items"}):
                list_links = [i.find('div',attrs={'class':"row"}).find('div',attrs={'class':"col-md-10 col-sm-10 col-xs-10"}) for i in soup.find_all('div',attrs={'class':"list-items"}) if i is not None]
                links_page = [i.h5.a.get('href') for i in list_links if i is not None]
                data_links.append(links_page)
                break
                
        except:
            print('Failed')

Failed
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
200
Failed
Failed
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
200
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
200
Failed
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
200
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
200
Failed
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
200
200
Failed
Failed
Failed
200
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
F

In [23]:
resoaldo = data_links.copy()

In [34]:
for i in data_links:
    print(i['City'])
    #print([d for c in i['Data'] for d in c])

abu-dhabi
ajman
al-ain
dubai
sharjah


In [39]:
last_dicts_city = []
for i in data_links:
    new_dict = {}
    new_dict['City'] = i['City']
    new_dict['Data'] = [d for c in i['Data'] for d in c]
    last_dicts_city.append(new_dict)

In [42]:
for i in last_dicts_city:
    print(i['Data'])

['https://www.edarabia.com/6186/horizon-private-school-khalifa-city/', 'https://www.edarabia.com/137352/virginia-international-private-school-abu-dhabi-uae/', 'https://www.edarabia.com/259/al-dhafra-private-school/', 'https://www.edarabia.com/137366/global-indian-international-school-abu-dhabi-uae/', 'https://www.edarabia.com/6388/british-international-school-abu-dhabi-uae/', 'https://www.edarabia.com/82124/cranleigh-abu-dhabi-school/', 'https://www.edarabia.com/112902/reach-british-school-abu-dhabi-uae/', 'https://www.edarabia.com/aspen-heights-british-school-ahbs-abu-dhabi-uae/', 'https://www.edarabia.com/93730/repton-school-abu-dhabi/', 'https://www.edarabia.com/6391/british-school-al-khubairat/', 'https://www.edarabia.com/21626/gems-american-academy-abu-dhabi/', 'https://www.edarabia.com/22370/liwa-international-school-abu-dhabi-uae/', 'https://www.edarabia.com/spanish-school-abu-dhabi-uae/', 'https://www.edarabia.com/4801/al-muna-primary-school/', 'https://www.edarabia.com/4813/al

In [ ]:
link_eua = list(set([a for i in data_links for a in i]))

In [69]:
last_dicts_city

[{'City': 'abu-dhabi',
  'Data': ['https://www.edarabia.com/6186/horizon-private-school-khalifa-city/',
   'https://www.edarabia.com/137352/virginia-international-private-school-abu-dhabi-uae/',
   'https://www.edarabia.com/259/al-dhafra-private-school/',
   'https://www.edarabia.com/137366/global-indian-international-school-abu-dhabi-uae/',
   'https://www.edarabia.com/6388/british-international-school-abu-dhabi-uae/',
   'https://www.edarabia.com/82124/cranleigh-abu-dhabi-school/',
   'https://www.edarabia.com/112902/reach-british-school-abu-dhabi-uae/',
   'https://www.edarabia.com/aspen-heights-british-school-ahbs-abu-dhabi-uae/',
   'https://www.edarabia.com/93730/repton-school-abu-dhabi/',
   'https://www.edarabia.com/6391/british-school-al-khubairat/',
   'https://www.edarabia.com/21626/gems-american-academy-abu-dhabi/',
   'https://www.edarabia.com/22370/liwa-international-school-abu-dhabi-uae/',
   'https://www.edarabia.com/spanish-school-abu-dhabi-uae/',
   'https://www.edara

# Part II

In [76]:
#data_info = []
#data_r = []

for idx, j in enumerate(last_dicts_city[4:]):
    list_links = list(set(j['Data']))
    print(f'{idx}/{len(last_dicts_city)}')
    num_proxy = 0
    for idx_l,link in enumerate(list_links):
        print(f'{idx_l}/{len(list_links)}')
        if len(lines) == num_proxy:
            print('Se agotaron los proxies')
        for idx_p, proxy in enumerate(lines[num_proxy:]):
            idx_p += num_proxy
            proxy = proxy.replace('\n','')
            try:
                r = requests.get(link,proxies={'http':proxy,'https':proxy},timeout=3)
                soup = BeautifulSoup(r.text,'lxml')
                school_name = soup.find('h1').text
                try:
                    leadership = soup.body.find(text="Leadership: ").find_parent('li').text.replace('Leadership:','').strip()
                except:
                    leadership = None
                address = remove_blank_spaces(soup.find(text="Address: ").find_parent('li').text.split('\n')[0].replace('(Map)','').replace('Address:',''))
                try:
                    tel = remove_blank_spaces(soup.find(text="Tel: ").find_parent('li').text.replace('Show Number','').replace('Tel:',''))
                except:
                    tel = None
                try:
                    website = soup.find('a',attrs={'title':"Visit Website"}).get('href')
                except:
                    website = None
                data = {
                'school_name':school_name,
                'Leadership':leadership,
                'Address':address,
                'Tel':tel,
                'Website': website,
                'link':link,
                'City':j['City']}
                data_info_copy.append(data)
                data_r.append(r)
                print('\tSuccess')
                num_proxy = idx_p
                break
        
            except ConnectionError as cx:
                pass
        
            except Exception as ex:
                print(ex)
                #traceback.print_exc(link)
                pass
        print('Success B')

0/5
0/25
'NoneType' object has no attribute 'find_parent'
	Success
Success B
1/25
	Success
Success B
2/25
	Success
Success B
3/25
	Success
Success B
4/25
'NoneType' object has no attribute 'find_parent'
'NoneType' object has no attribute 'find_parent'
Success B
5/25
	Success
Success B
6/25
	Success
Success B
7/25
	Success
Success B
8/25
	Success
Success B
9/25
	Success
Success B
10/25
'NoneType' object has no attribute 'find_parent'
'NoneType' object has no attribute 'find_parent'
Success B
11/25
Success B
12/25
Success B
13/25
Success B
14/25
Success B
15/25
Success B
16/25
Success B
17/25
Success B
18/25
Success B
19/25
Success B
20/25
Success B
21/25
Success B
22/25
Success B
23/25
Success B
24/25
Success B
1/5
0/25
Success B
1/25
Success B
2/25
Success B
3/25
Success B
4/25
Success B
5/25
Success B
6/25
Success B
7/25
HTTPSConnectionPool(host='www.edarabia.com', port=443): Read timed out. (read timeout=3)
Success B
8/25
'NoneType' object has no attribute 'find_parent'
'NoneType' ob

In [11]:
#data_info = []
#data_r = []

for idx, link in enumerate(link_eua):
    print(f'{idx}/{len(link_eua)}')
    for proxy in lines:
        proxy = i.replace('\n','')
        try:
            r = requests.get(link,proxies={'http':proxy,'https':proxy},timeout=3)
            soup = BeautifulSoup(r.text,'lxml')
            school_name = soup.find('h1').text
            try:
                leadership = soup.body.find(text="Leadership: ").find_parent('li').text.replace('Leadership:','').strip()
            except:
                leadership = None
            address = remove_blank_spaces(soup.find(text="Address: ").find_parent('li').text.split('\n')[0].replace('(Map)','').replace('Address:',''))
            try:
                tel = remove_blank_spaces(soup.find(text="Tel: ").find_parent('li').text.replace('Show Number','').replace('Tel:',''))
            except:
                tel = None
            try:
                website = soup.find('a',attrs={'title':"Visit Website"}).get('href')
            except:
                website = None
            data = {
            'school_name':school_name,
            'Leadership':leadership,
            'Address':address,
            'Tel':tel,
            'Website': website,
            'link':link}
            data_info.append(data)
            data_r.append(r)
            break
            
        except Exception as ex:
            print(ex)
            print(link)
            pass

0/544
1/544
2/544
3/544
4/544
5/544
HTTPSConnectionPool(host='www.edarabia.com', port=443): Max retries exceeded with url: /113541/oasis-international-school-al-ain-uae/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f90679c7e90>, 'Connection to 185.190.141.11 timed out. (connect timeout=3)'))
https://www.edarabia.com/113541/oasis-international-school-al-ain-uae/
HTTPSConnectionPool(host='www.edarabia.com', port=443): Max retries exceeded with url: /113541/oasis-international-school-al-ain-uae/ (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out')))
https://www.edarabia.com/113541/oasis-international-school-al-ain-uae/
6/544
HTTPSConnectionPool(host='www.edarabia.com', port=443): Max retries exceeded with url: /963/al-ain-juniors-school/ (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out')))
https://www.edarabia.com/963/al-ain-juniors-school/
7/544
8/544
9/544
10/544
11/544
12/544
13/544
14/544
15/544
16/544
17/544
18/

In [56]:
data_info_copy = data_info.copy()

In [77]:
data_info = data_info_copy

In [78]:
for idx, i in enumerate(data_info):
    n = 0
    try:
        data_info[idx] = pd.DataFrame(i)
    except ValueError as e:
        data_info[idx] = pd.DataFrame.from_dict(i,orient='index').transpose()

In [79]:
data_info = pd.concat(data_info)

In [80]:
data_info.drop_duplicates(inplace=True)

In [81]:
data_info.shape

(100, 7)

In [60]:
data_info

,school_name,Leadership,Address,Tel,Website,link,City
0,"Horizon Private School- Branch, Khalifa City",Ms. Amal Shahine (Principal),Khalifa City A Street 26 - Abu Dhabi UAE,+97125560811,http://www.horizonprivateschool.com,https://www.edarabia.com/6186/horizon-private-...,abu-dhabi
0,Virginia International Private School,Daniel Bokelman (Principal),MFW-31 Shakhbout City - Abu Dhabi UAE,+97125554383,http://virginiaschool.ae/en/,https://www.edarabia.com/137352/virginia-inter...,abu-dhabi
0,Al Dhafra Private Schools - Abu Dhabi,Mr. Principal: David Craig Evans,ME09 Mohamed Bin Zayed City - Abu Dhabi UAE,+97126108400401,http://www.dhafraschools.com,https://www.edarabia.com/259/al-dhafra-private...,abu-dhabi
0,"Global Indian International School, Abu Dhabi",Dr. Heena Rachh (Principal),Baniyas East Behind Lulu Hypermarket - Abu Dha...,+971522181585,http://abudhabi.globalindianschool.org,https://www.edarabia.com/137366/global-indian-...,abu-dhabi
0,"British International School, Abu Dhabi",Patrick Horne (Principal),Behind Abu Dhabi University off E22 Al Ain Roa...,+97125100100,http://bit.ly/3uOHCAm,https://www.edarabia.com/6388/british-internat...,abu-dhabi
...,...,...,...,...,...,...,...
0,"Repton School, Dubai",David Cook (Headmaster),Nad Al Sheba 3 - Dubai UAE,+97144269393,http://www.reptondubai.org,https://www.edarabia.com/673/repton-school-dubai/,dubai
0,Newlands School Dubai,Mr. Matthew Edwards (Principal),Al Warqa 1 - Dubai UAE,+97142821200,http://www.newlandsschool.ae,https://www.edarabia.com/newlands-school-dubai...,dubai
0,"GEMS Wellington Academy, Silicon Oasis",Mr. Principal: Kevin Loft,Silicon Oasis - Dubai UAE,+97145159000,http://www.gemswellingtonacademy-dso.com,https://www.edarabia.com/21614/gems-wellington...,dubai
0,"Emirates International School, Meadows",Kathryn Dyche - Nichols (Principal),Emirates Living The Meadows - Dubai UAE,+97143629009,http://www.eischools.ae/Meadows,https://www.edarabia.com/575/emirates-internat...,dubai


In [65]:
data_info.sample(10)

,school_name,Leadership,Address,Tel,Website,link,City
0,The First Academy Ajman,Dr. Abdul Rahman Alnuaimi (Chairman),Al Hamidia Opp Health Center 2001 - Ajman UAE,+97167417111,http://www.firstacademy.org/index.php/en/,https://www.edarabia.com/105765/the-first-acad...,ajman
0,Al Ain Khaleej Private School,Raed Al Qasrawi (Principal),Fallaj Hazzaa - Al Ain UAE,+97137811232,http://ainalkhaleejprivateschools.com,https://www.edarabia.com/6436/al-ain-khaleej-p...,al-ain
0,Indian School Ajman,Ms. Malathi Das (Principal),Sheikh Maktoum Bin Rashid StreetAl Jurf - Ajma...,+97167437477,http://www.indianschoolajman.org,https://www.edarabia.com/105700/indian-school-...,ajman
0,Shining Star International School Abu Dhabi,Mrs. Abhilasha Singh (Principal),Shabiya 12 Musaffah - Abu Dhabi UAE,+97125599080 / +97125596444,http://www.ssis.ae,https://www.edarabia.com/112905/shining-star-i...,abu-dhabi
0,Abu Dhabi Island International Private School,Brendan Aspell (Principal),Al Towayya - Al Ain UAE,+97137227666,http://www.adiips.org,https://www.edarabia.com/120549/abu-dhabi-isla...,al-ain
0,International Indian School Ajman,Mrs. Qurat Al Ain (Principal),Al Jurf 2 - Ajman UAE,+97167408333,http://www.iisajman.org,https://www.edarabia.com/105704/international-...,ajman
0,Credence High School Dubai,Ms. Deepika Thapar Singh (Principal),Al Quoz 4 - Dubai UAE,+97143212144,http://www.credencehighschool.com,https://www.edarabia.com/113918/credence-high-...,dubai
0,Liwa International School Al Mushrif,Mrs. Margaret Haynes (Principal),22 Um Salamah St Al Mushrif - Abu Dhabi UAE,+97124441301,http://www.lism.ae,https://www.edarabia.com/22370/liwa-internatio...,abu-dhabi
0,Nord Anglia International School Dubai,Matthew Farthing (Principal),Al Barsha South Sheikh Zayed road Hessa Street...,+97142199999,http://www.nordangliaeducation.com/our-schools...,https://www.edarabia.com/93989/nord-anglia-int...,dubai
0,Zakher Private School,Ms. Agnes Nathaniel (Head),Falaj Hazaa - Al Ain UAE,+97137810818,http://www.zakherprivateschool.com,https://www.edarabia.com/6528/zakher-private-s...,al-ain


In [82]:
df = data_info.drop(columns=['link'])

In [ ]:
daa

In [83]:
df.to_excel('/home/cesar/Documents/job/web_scraping/cg_wilson/edarabia_cities.xlsx',index=False)

In [52]:
with open('../../cg_wilson/new_dict.dat','rb') as f:
    info_r = pickle.load(f)

In [59]:
info_r[0]['Data']

[['https://www.edarabia.com/6186/horizon-private-school-khalifa-city/',
  'https://www.edarabia.com/137352/virginia-international-private-school-abu-dhabi-uae/',
  'https://www.edarabia.com/259/al-dhafra-private-school/',
  'https://www.edarabia.com/137366/global-indian-international-school-abu-dhabi-uae/',
  'https://www.edarabia.com/6388/british-international-school-abu-dhabi-uae/',
  'https://www.edarabia.com/82124/cranleigh-abu-dhabi-school/',
  'https://www.edarabia.com/112902/reach-british-school-abu-dhabi-uae/',
  'https://www.edarabia.com/aspen-heights-british-school-ahbs-abu-dhabi-uae/',
  'https://www.edarabia.com/93730/repton-school-abu-dhabi/',
  'https://www.edarabia.com/6391/british-school-al-khubairat/',
  'https://www.edarabia.com/21626/gems-american-academy-abu-dhabi/',
  'https://www.edarabia.com/22370/liwa-international-school-abu-dhabi-uae/',
  'https://www.edarabia.com/spanish-school-abu-dhabi-uae/',
  'https://www.edarabia.com/4801/al-muna-primary-school/',
  'htt

In [60]:
last_dicts_city = []
for i in info_r:
    new_dict = {}
    new_dict['City'] = i['City']
    new_dict['Data'] = [d for c in i['Data'] for d in c]
    last_dicts_city.append(new_dict)

In [99]:
def get_urls(dict_list):
    url = dict_list['Data']
    return url

def parse(lst_link,proxy):
    reqs = [grequests.get(link,proxies={'http':proxy,'https':proxy},timeout=3,headers={'user-agent':'Mozilla/5.0 (X11; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0'}) for link in lst_link]
    reqs = grequests.map(reqs)
    return reqs

In [68]:
resp

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [100]:
data_info = []
data_r = []
for idx, j in enumerate(last_dicts_city):
    list_links = list(set(j['Data']))
    print(f'{idx}/{len(last_dicts_city)}')
    num_proxy = 0
    if len(lines) == num_proxy:
        print('Se agotaron los proxies')
    for idx_proxy , proxy in enumerate(lines[num_proxy:]):
        num_proxy += 1
        proxy = proxy.replace('\n','')
        try:
            resp = parse(list_links,proxy)
            for r in resp:
                if r == None:
                    raise ConnectionError('R is None')
                soup = BeautifulSoup(r.text,'lxml')
                school_name = soup.find('h1').text
                print(school_name)
                try:
                    leadership = soup.body.find(text="Leadership: ").find_parent('li').text.replace('Leadership:','').strip()
                except:
                    leadership = None
                address = remove_blank_spaces(soup.find(text="Address: ").find_parent('li').text.split('\n')[0].replace('(Map)','').replace('Address:',''))
                try:
                    tel = remove_blank_spaces(soup.find(text="Tel: ").find_parent('li').text.replace('Show Number','').replace('Tel:',''))
                except:
                    tel = None
                try:
                    website = soup.find('a',attrs={'title':"Visit Website"}).get('href')
                except:
                    website = None
                data = {
                'school_name':school_name,
                'Leadership':leadership,
                'Address':address,
                'Tel':tel,
                'Website': website,
                #'link':link,
                'City':j['City']}
                data_info.append(data)
                data_r.append(r)
                print('\tSuccess')
                num_proxy = idx_proxy
                break
        
        except ConnectionError as cx:
            #pass
            #rint(num_proxy)
            pass
        
        except Exception as ex:
            print(ex)
            #print(link)
            pass
    print('Success B')

0/193
Cranleigh Abu Dhabi School 
	Success
Please Enable Cookies
'NoneType' object has no attribute 'find_parent'
Please Enable Cookies
'NoneType' object has no attribute 'find_parent'
Cranleigh Abu Dhabi School 
	Success
Please Enable Cookies
'NoneType' object has no attribute 'find_parent'
Success B
1/193
City School, Ajman 
	Success
Please Enable Cookies
'NoneType' object has no attribute 'find_parent'
Success B
2/193
Please Enable Cookies
'NoneType' object has no attribute 'find_parent'
Please Enable Cookies
'NoneType' object has no attribute 'find_parent'


KeyboardInterrupt
2022-03-11T13:24:19Z


KeyboardInterrupt: 

In [101]:
data_info

[{'school_name': 'Cranleigh Abu Dhabi School ',
  'Leadership': 'Mr. Michael Wilson (Principal)',
  'Address': 'St Regis Residences Saadiyat - Abu Dhabi UAE',
  'Tel': '+97124970000',
  'Website': 'http://www.cranleigh.ae',
  'City': 'abu-dhabi'},
 {'school_name': 'Cranleigh Abu Dhabi School ',
  'Leadership': 'Mr. Michael Wilson (Principal)',
  'Address': 'St Regis Residences Saadiyat - Abu Dhabi UAE',
  'Tel': '+97124970000',
  'Website': 'http://www.cranleigh.ae',
  'City': 'abu-dhabi'},
 {'school_name': 'City School, Ajman ',
  'Leadership': 'Richard Monteiro (Principal)',
  'Address': 'Sheikh Ammar bin Humaid Road Al Tallah 2 - Ajman UAE',
  'Tel': '+97167147777',
  'Website': 'http://cityschool.ae',
  'City': 'ajman'}]

# Check Proxies

In [102]:
with open('/home/cesar/Downloads/http_proxies.txt', 'r') as f:
    src_proxy_names = [line.strip() for line in f if line.strip()]

def make_check_req(proxy):
    res = grequests.get(a[0],
                        proxies={ 'http': proxy, 'https':proxy },
                        headers={'x-my-proxy' : proxy },
                        timeout=20)
    return res

def get_proxy_if_good(res):
    if res and res.status_code == 200:
        return tuple(res.request.headers['x-my-proxy'].split(' '))

proxy_check_requests = map(make_check_req, src_proxy_names)

In [103]:
proxy_check_results = grequests.map(proxy_check_requests)

In [104]:
good_proxies = list(set(filter(None, map(get_proxy_if_good, proxy_check_results))))
print('Number of good proxies', len(good_proxies))
with open('/home/cesar/Downloads/good_proxies.txt', 'w') as f:
    f.write('\n'.join('\t'.join(t) for t in good_proxies))

Number of good proxies 28


In [46]:
! pwd

/home/cesar/Documents/job/web_scraping/javier/agenda


In [18]:
for proxy in lines[20:]:
    rs = (grequests.get(u, proxies={'http':proxy,'https':proxy},timeout=3) for u in a)
    rs = grequests.map(rs)
    try:
        if rs.status_code == 200:
            print('Bueno')
    except:
        print(rs)
        pass

[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[<Response [200]>]
[None]
[None]
[<Response [200]>]
[None]
[None]
[None]
[None]


KeyboardInterrupt
2022-03-11T04:38:32Z


KeyboardInterrupt: 

In [47]:
rs

<generator object <genexpr> at 0x7f469f9fcdd0>

In [8]:
rs = (grequests.get(u, proxies={'http':proxy,'https':proxy},timeout=3) for u in a)

In [9]:
grequests.map(rs)

[None]

In [158]:
df = pd.read_csv('/home/cesar/Documents/job/web_scraping/mapandmarket/inmobiliaria/results_inmobiliaria_22_4.csv')
df.head()

,Nombre,Location,Price,Metros cuadrados,Numero de Baños,Numero de habitaciones,Espacio Disponible,Eficiencia Electrica,precio por metro,Province,City,Images,Descripcion,Tipo
0,Unifamiliar en venta en Sabiñanigo de 108 m²,"c. marques de urquijo, Sabiñanigo, Huesca",49.900€,108 m²,1 bañ.,2 hab.,2 habitaciones 1 baño,En tramite,108 m² - 462 €/m²,Aragón,Sabiñánigo,https://u.inmobiliariabancaria.com/viviendas/2...,"¡Promoción de viviendas en Sabiñanigo, Huesca!",casas-bancos
1,Unifamiliar en venta en Villanueva De Tapia de...,"CONSTITUCION, Villanueva de tapia, Malaga",58.700€,189 m²,2 bañ.,4 hab.,4 habitaciones 2 baños,En tramite,189 m² - 311 €/m²,Andalucía,Villanueva de Tapia,https://u.inmobiliariabancaria.com/viviendas/2...,Casa adosada con 4 habitaciones en la provinci...,casas-bancos
2,Unifamiliar en venta en Villanueva De Tapia de...,"SAN JOSE, Villanueva de tapia, Malaga",45.000€,155 m²,1 bañ.,3 hab.,3 habitaciones 1 baño,E,155 m² - 290 €/m²,Andalucía,Villanueva de Tapia,https://u.inmobiliariabancaria.com/viviendas/2...,Casa adosada con 3 habitaciones en la provinci...,casas-bancos
3,Unifamiliar en venta en Huetor Vega de 434 m²,"c. granada, Huetor Vega, Granada",237.500€,434 m²,4 bañ.,5 hab.,5 habitaciones 4 baños,E,434 m² - 547 €/m²,Andalucía,Huétor Vega,https://u.inmobiliariabancaria.com/viviendas/2...,Casa de pueblo Señorial distribuida en 2 plant...,casas-bancos
4,Unifamiliar en venta en Itrabo de 140 m²,"c. era, Itrabo, Granada",65.000€,140 m²,NaN,NaN,NaN,E,140 m² - 464 €/m²,Andalucía,Itrabo,https://u.inmobiliariabancaria.com/viviendas/2...,"Vivienda situada en Itrabo, Granada. Ubicada e...",casas-bancos


In [159]:
df = df[df['Price'] != 'A consultar']

In [160]:
df['main_image'] = df.Images.apply(lambda x : x.split(',')[0])

In [161]:
df['Tipo'] = df.Tipo.apply(lambda x : x.replace('-bancos','').capitalize())

In [162]:
df.head()

,Nombre,Location,Price,Metros cuadrados,Numero de Baños,Numero de habitaciones,Espacio Disponible,Eficiencia Electrica,precio por metro,Province,City,Images,Descripcion,Tipo,main_image
0,Unifamiliar en venta en Sabiñanigo de 108 m²,"c. marques de urquijo, Sabiñanigo, Huesca",49.900€,108 m²,1 bañ.,2 hab.,2 habitaciones 1 baño,En tramite,108 m² - 462 €/m²,Aragón,Sabiñánigo,https://u.inmobiliariabancaria.com/viviendas/2...,"¡Promoción de viviendas en Sabiñanigo, Huesca!",Casas,https://u.inmobiliariabancaria.com/viviendas/2...
1,Unifamiliar en venta en Villanueva De Tapia de...,"CONSTITUCION, Villanueva de tapia, Malaga",58.700€,189 m²,2 bañ.,4 hab.,4 habitaciones 2 baños,En tramite,189 m² - 311 €/m²,Andalucía,Villanueva de Tapia,https://u.inmobiliariabancaria.com/viviendas/2...,Casa adosada con 4 habitaciones en la provinci...,Casas,https://u.inmobiliariabancaria.com/viviendas/2...
2,Unifamiliar en venta en Villanueva De Tapia de...,"SAN JOSE, Villanueva de tapia, Malaga",45.000€,155 m²,1 bañ.,3 hab.,3 habitaciones 1 baño,E,155 m² - 290 €/m²,Andalucía,Villanueva de Tapia,https://u.inmobiliariabancaria.com/viviendas/2...,Casa adosada con 3 habitaciones en la provinci...,Casas,https://u.inmobiliariabancaria.com/viviendas/2...
3,Unifamiliar en venta en Huetor Vega de 434 m²,"c. granada, Huetor Vega, Granada",237.500€,434 m²,4 bañ.,5 hab.,5 habitaciones 4 baños,E,434 m² - 547 €/m²,Andalucía,Huétor Vega,https://u.inmobiliariabancaria.com/viviendas/2...,Casa de pueblo Señorial distribuida en 2 plant...,Casas,https://u.inmobiliariabancaria.com/viviendas/2...
4,Unifamiliar en venta en Itrabo de 140 m²,"c. era, Itrabo, Granada",65.000€,140 m²,NaN,NaN,NaN,E,140 m² - 464 €/m²,Andalucía,Itrabo,https://u.inmobiliariabancaria.com/viviendas/2...,"Vivienda situada en Itrabo, Granada. Ubicada e...",Casas,https://u.inmobiliariabancaria.com/viviendas/2...


In [163]:
df.Province = df.Location.apply(lambda x : x.split(',')[-1].strip())

In [164]:
df.Province.unique()

array(['Huesca', 'Malaga', 'Granada', 'Vizcaya', 'Cadiz', 'Valencia',
       'Jaen', 'Almeria', 'Huelva', 'Castellon', 'Cantabria', 'Guipuzcoa',
       'Alava', 'Asturias', 'Navarra', 'Alicante', 'Madrid', 'Murcia',
       'Illes Balears', 'A Coruna', 'Lugo', 'Badajoz', 'Girona',
       'Ourense', 'Caceres', 'Lleida', 'Tarragona', 'Pontevedra',
       'La Rioja', 'Santa Cruz De Tenerife', 'Las Palmas', 'Barcelona',
       'Toledo', 'Guadalajara', 'Cuenca', 'Ciudad Real', 'Albacete',
       'Valladolid', 'Palencia', 'Salamanca', 'Teruel', 'Segovia', 'Leon',
       'Zaragoza', 'Avila', 'Burgos', 'Zamora', 'Cordoba', 'Sevilla',
       'Soria'], dtype=object)

In [149]:
df[df['Province'] == 'Badajoz']

,Nombre,Location,Price,Metros cuadrados,Numero de Baños,Numero de habitaciones,Espacio Disponible,Eficiencia Electrica,precio por metro,Province,City,Images,Descripcion,Tipo,main_image
3256,Nave en venta en Fregenal De La Sierra de 1199...,"c. b - poligono industrial frexnense), Fregena...",231.600€,1199 m²,NaN,NaN,NaN,En tramite,1199 m² - 193 €/m²,Badajoz,Fregenal de la Sierra,https://u.inmobiliariabancaria.com/viviendas/2...,Nave en dos alturas que consta de planta semis...,Naves-industriales,https://u.inmobiliariabancaria.com/viviendas/2...
3258,Nave en venta en Don Benito,"Calle Arenas, Don Benito, Badajoz",110.800€,NaN,NaN,NaN,NaN,En tramite,NaN,Badajoz,Don Benito,/imagen/foto-piso-no-encontrada.jpg,Nave de segunda mano ubicada en la calle Arena...,Naves-industriales,/imagen/foto-piso-no-encontrada.jpg
3265,Nave en venta en Castuera de 821 m²,"Polígono Montepozuelo-, Castuera, Badajoz",249.575€,821 m²,NaN,NaN,NaN,En tramite,821 m² - 304 €/m²,Badajoz,Castuera,https://u.inmobiliariabancaria.com/viviendas/2...,"Nave de reciente construcción, de 2 plantas, e...",Naves-industriales,https://u.inmobiliariabancaria.com/viviendas/2...
3267,Nave en venta en Badajoz de 1760 m²,"avda. felipe trigo, Badajoz, Badajoz",741.000€,1760 m²,NaN,NaN,NaN,En tramite,1760 m² - 421 €/m²,Badajoz,Badajoz,https://u.inmobiliariabancaria.com/viviendas/2...,Nave industrial compuesta de 2 plantas y con a...,Naves-industriales,https://u.inmobiliariabancaria.com/viviendas/2...
3269,Nave en venta en Almendralejo de 2930 m²,"c. vapor, Almendralejo, Badajoz",700.900€,2930 m²,NaN,NaN,NaN,En tramite,2930 m² - 239 €/m²,Badajoz,Almendralejo,https://u.inmobiliariabancaria.com/viviendas/2...,Nave industrial de dos plantas y ubicada en un...,Naves-industriales,https://u.inmobiliariabancaria.com/viviendas/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69702,Unifamiliar en venta en Badajoz de 112 m²,"r - casas aisladas, Badajoz, Badajoz",92.000€,112 m²,NaN,NaN,NaN,D,112 m² - 821 €/m²,Badajoz,Badajoz,/imagen/foto-piso-no-encontrada.jpg,-,Pisos,/imagen/foto-piso-no-encontrada.jpg
69703,Unifamiliar en venta en Badajoz de 55 m²,"c. diego florindo, Badajoz, Badajoz",75.200€,55 m²,2 bañ.,4 hab.,4 habitaciones 2 baños,En tramite,55 m² - 1367 €/m²,Badajoz,Badajoz,https://u.inmobiliariabancaria.com/viviendas/2...,Promoción de viviendas y plazas de parking en ...,Pisos,https://u.inmobiliariabancaria.com/viviendas/2...
69705,Unifamiliar en venta en Badajoz de 288 m²,"Calle Concepción Arenal, Badajoz, Badajoz",81.400€,288 m²,1 bañ.,3 hab.,3 habitaciones 1 baño,En tramite,288 m² - 283 €/m²,Badajoz,Badajoz,https://u.inmobiliariabancaria.com/viviendas/2...,Vivienda adosada en Badajoz. Cuenta con una su...,Pisos,https://u.inmobiliariabancaria.com/viviendas/2...
69708,Unifamiliar en venta en Badajoz de 104 m²,"c. luis zambrano blanco, Badajoz, Badajoz",104.000€,104 m²,2 bañ.,3 hab.,3 habitaciones 2 baños,En tramite,104 m² - 1000 €/m²,Badajoz,Badajoz,https://u.inmobiliariabancaria.com/viviendas/2...,Viviendas con plaza de parking con trastero.,Pisos,https://u.inmobiliariabancaria.com/viviendas/2...


In [165]:
df_ba = df[df.Province == 'Badajoz']

In [166]:
df_ba = df_ba.dropna()

In [167]:
df_ba['Price'] = df_ba.Price.apply(lambda x : x.replace('.',''))

In [168]:
df_ba.head()

,Nombre,Location,Price,Metros cuadrados,Numero de Baños,Numero de habitaciones,Espacio Disponible,Eficiencia Electrica,precio por metro,Province,City,Images,Descripcion,Tipo,main_image
7765,Local en venta en Almendralejo de 125 m²,"c. escribano, Almendralejo, Badajoz",75500€,125 m²,1 bañ.,2 hab.,2 habitaciones 1 baño,En tramite,125 m² - 604 €/m²,Badajoz,Almendralejo,https://u.inmobiliariabancaria.com/viviendas/2...,Magníficas viviendas a estrenar en el centro d...,Locales-venta,https://u.inmobiliariabancaria.com/viviendas/2...
7785,Local en venta en Almendralejo de 125 m²,"c. santa marta, Almendralejo, Badajoz",130000€,125 m²,2 bañ.,2 hab.,2 habitaciones 2 baños,En tramite,125 m² - 1040 €/m²,Badajoz,Almendralejo,https://u.inmobiliariabancaria.com/viviendas/2...,"Pisos de 1 o 2 habitaciones y 1 o 2 baños, los...",Locales-venta,https://u.inmobiliariabancaria.com/viviendas/2...
8143,Local en venta en Merida de 311 m²,"avda. reina sofia, Merida, Badajoz",92000€,311 m²,2 bañ.,3 hab.,3 habitaciones 2 baños,En tramite,311 m² - 296 €/m²,Badajoz,Mérida,https://u.inmobiliariabancaria.com/viviendas/2...,"Viviendas, oficinas y locales en Mérida. \r\n\...",Locales-venta,https://u.inmobiliariabancaria.com/viviendas/2...
13410,Chalet en venta en Mérida de 132 m²,"Calle Camino de la Magdalena, Mérida, Badajoz",50600€,132 m²,1 bañ.,4 hab.,4 habitaciones 1 baño,En tramite,132 m² - 383 €/m²,Badajoz,Mérida,/imagen/foto-piso-no-encontrada.jpg,Encuentra Pisos de Segunda Mano en Mérida. Sol...,Chalet,/imagen/foto-piso-no-encontrada.jpg
29256,Piso en venta en Valdelacalzada de 198 m²,"c. puebla, Valdelacalzada, Badajoz",47300€,198 m²,1 bañ.,5 hab.,5 habitaciones 1 baño,En tramite,198 m² - 239 €/m²,Badajoz,Valdelacalzada,/imagen/foto-piso-no-encontrada.jpg,-,Pisos-venta,/imagen/foto-piso-no-encontrada.jpg


In [169]:
df = df[df.Province != 'Badajoz'] 

In [170]:
df = df.dropna()

In [171]:
df['Price'] = df.Price.apply(lambda x : x.replace('.',''))

In [172]:
df.head()

,Nombre,Location,Price,Metros cuadrados,Numero de Baños,Numero de habitaciones,Espacio Disponible,Eficiencia Electrica,precio por metro,Province,City,Images,Descripcion,Tipo,main_image
0,Unifamiliar en venta en Sabiñanigo de 108 m²,"c. marques de urquijo, Sabiñanigo, Huesca",49900€,108 m²,1 bañ.,2 hab.,2 habitaciones 1 baño,En tramite,108 m² - 462 €/m²,Huesca,Sabiñánigo,https://u.inmobiliariabancaria.com/viviendas/2...,"¡Promoción de viviendas en Sabiñanigo, Huesca!",Casas,https://u.inmobiliariabancaria.com/viviendas/2...
1,Unifamiliar en venta en Villanueva De Tapia de...,"CONSTITUCION, Villanueva de tapia, Malaga",58700€,189 m²,2 bañ.,4 hab.,4 habitaciones 2 baños,En tramite,189 m² - 311 €/m²,Malaga,Villanueva de Tapia,https://u.inmobiliariabancaria.com/viviendas/2...,Casa adosada con 4 habitaciones en la provinci...,Casas,https://u.inmobiliariabancaria.com/viviendas/2...
2,Unifamiliar en venta en Villanueva De Tapia de...,"SAN JOSE, Villanueva de tapia, Malaga",45000€,155 m²,1 bañ.,3 hab.,3 habitaciones 1 baño,E,155 m² - 290 €/m²,Malaga,Villanueva de Tapia,https://u.inmobiliariabancaria.com/viviendas/2...,Casa adosada con 3 habitaciones en la provinci...,Casas,https://u.inmobiliariabancaria.com/viviendas/2...
3,Unifamiliar en venta en Huetor Vega de 434 m²,"c. granada, Huetor Vega, Granada",237500€,434 m²,4 bañ.,5 hab.,5 habitaciones 4 baños,E,434 m² - 547 €/m²,Granada,Huétor Vega,https://u.inmobiliariabancaria.com/viviendas/2...,Casa de pueblo Señorial distribuida en 2 plant...,Casas,https://u.inmobiliariabancaria.com/viviendas/2...
5,Unifamiliar en venta en Alhaurin El Grande de ...,"PTO.GOMEZ, Alhaurin el grande, Malaga",137600€,152 m²,2 bañ.,2 hab.,2 habitaciones 2 baños,En tramite,152 m² - 905 €/m²,Malaga,Alhaurín el Grande,https://u.inmobiliariabancaria.com/viviendas/2...,CON INCIDENCIA - Casa con 2 habitaciones en la...,Casas,https://u.inmobiliariabancaria.com/viviendas/2...


In [173]:
df_cas = df[df['Province'] == 'Castellon']

In [174]:
df = df[df['Province'] != 'Castellón de la Plana']

In [175]:
df.to_csv('/home/cesar/Desktop/results_inmobiliaria_20_4_f.csv', index=False)

In [177]:
df_ba.to_csv('/home/cesar/Desktop/results_inmobiliaria_20_4_ba.csv', index=False)

In [176]:
df_cas.to_csv('/home/cesar/Desktop/results_inmobiliaria_20_4_cas.csv', index=False)

In [121]:
list(df_ba[df_ba['Nombre'] == 'Atico en venta en Badajoz de 119  m²'].main_image)[0]

'https://u.inmobiliariabancaria.com/viviendas/202108/729053/5131504/imagen-8752431.jpg'

In [115]:
list(df_ba['Nombre'])[-1]

'Unifamiliar en venta en Badajoz de 157  m²'

In [6]:
test = pd.read_csv('/home/cesar/Desktop/results_inmobiliaria_20_4_cas.csv').iloc[:2,:]

In [7]:
test.to_csv('/home/cesar/Desktop/test.csv')

In [21]:
df = pd.read_csv('/home/cesar/Documents/job/web_scraping/dwmex2015/civico/results_civico_21_4.csv')
df.head()

,Nombre Empresa,Direccion,Telefonos,Email,Web,Nombre de la Página
0,Textiles Cuernitos Ajusco,Calle Héroes de Churubusco # 14,5.255137e+11,mailto:texcuernitos_ajusco@hotmail.com,https://www.civico.com/lugar/textiles-cuernito...,Civico
1,Mala Sangre Tattoo Crew,Bucareli 168 Local 3,5.255513e+11,mailto:viczrico@gmail.com,https://www.civico.com/lugar/mala-sangre-tatto...,Civico
2,Juguetes y didácticos King Lotery,On line,6.618241e+07,mailto:ivanchas@live.com.mx,https://www.civico.com/lugar/juguetes-y-didact...,Civico
3,Natural English,Av. Insurgentes Sur #670,5.255681e+11,mailto:ne.jorgemogollon@gmail.com,https://www.civico.com/lugar/natural-english-m...,Civico
4,Guitarras Chema,Ayuntamiento 28 - Mercado de Artesanías San Juan,5.255551e+11,mailto:enriquezguitarras@yahoo.com,https://www.civico.com/lugar/guitarras-chema-m...,Civico


In [22]:
df['Email'] = df['Email'].apply(lambda x : x.replace('mailto:',''))

In [25]:
df2 =  pd.read_csv('/home/cesar/Documents/job/web_scraping/dwmex2015/civico/results_civico_22_4.csv')
df2.head()

,Nombre Empresa,Direccion,Telefonos,Email,Web,Nombre de la Página
0,Distribuidora De Artículos De Temporada,Pradera Esquina Zabala 16,5.255713e+11,mailto:paragualosgemelos@gmail.com,https://www.civico.com/lugar/distribuidora-de-...,Civico
1,Mine,Rinconada De Los Ríos S/N,5.255165e+11,mailto:mine.fitnessprogram@gmail.com,https://www.civico.com/lugar/mine-mexico,Civico
2,Sitax Servicio De Taxi A Domicilio,Agustín Melgar S/N,5.255260e+11,mailto:sergiogalletitami@gmail.com,https://www.civico.com/lugar/sitax-servicio-de...,Civico
3,Lda System México,Calle Enrique González Martínez # 106-A,5.255283e+11,mailto:josejuan.cruz@ldasys.mx,https://www.civico.com/lugar/lda-system-mexico...,Civico
4,Frodo Tattoo Estudio,1ra Cda. De Lázaro Cárdenas #182,5.255388e+11,mailto:afy.chef.jac@gmail.com,https://www.civico.com/lugar/frodo-tattoo-estu...,Civico


In [26]:
df2['Email'] = df2['Email'].apply(lambda x : x.replace('mailto:',''))

In [28]:
df = pd.concat([df,df2])

In [32]:
df = df.drop_duplicates()

In [33]:
df.head()

,Nombre Empresa,Direccion,Telefonos,Email,Web,Nombre de la Página
0,Textiles Cuernitos Ajusco,Calle Héroes de Churubusco # 14,5.255137e+11,texcuernitos_ajusco@hotmail.com,https://www.civico.com/lugar/textiles-cuernito...,Civico
1,Mala Sangre Tattoo Crew,Bucareli 168 Local 3,5.255513e+11,viczrico@gmail.com,https://www.civico.com/lugar/mala-sangre-tatto...,Civico
2,Juguetes y didácticos King Lotery,On line,6.618241e+07,ivanchas@live.com.mx,https://www.civico.com/lugar/juguetes-y-didact...,Civico
3,Natural English,Av. Insurgentes Sur #670,5.255681e+11,ne.jorgemogollon@gmail.com,https://www.civico.com/lugar/natural-english-m...,Civico
4,Guitarras Chema,Ayuntamiento 28 - Mercado de Artesanías San Juan,5.255551e+11,enriquezguitarras@yahoo.com,https://www.civico.com/lugar/guitarras-chema-m...,Civico


In [34]:
df.to_excel('/home/cesar/Documents/job/web_scraping/dwmex2015/civico/results_civico_22_4.xlsx',index=False)